In [1]:
import codecs
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.utils import shuffle
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import datapath
import numpy as np
import argparse
import copy
import operator
import re
import json
#from scipy.special import softmax
use_cuda = torch.cuda.is_available()

In [2]:
def load_data(fpath):
    data = []
    with open(fpath) as json_file:
        json_data = json.load(json_file)
        for i in range(0,len(json_data)):
            string = json_data[i]["sentence"]
            string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
            string = re.sub(r"\'s", " \'s", string)
            string = re.sub(r"\'ve", " \'ve", string)
            string = re.sub(r"n\'t", " n\'t", string)
            string = re.sub(r"\'re", " \'re", string)
            string = re.sub(r"\'d", " \'d", string)
            string = re.sub(r"\'ll", " \'ll", string)
            string = re.sub(r",", " , ", string)
            string = re.sub(r"!", " ! ", string)
            string = re.sub(r"\(", " \( ", string)
            string = re.sub(r"\)", " \) ", string)
            string = re.sub(r"\?", " \? ", string)
            string = re.sub(r"\s{2,}", " ", string)
            string = string.rstrip()
            if json_data[i]["aspect"] == 'food':
                data.append((string.split(' '),0))
            if json_data[i]["aspect"] == 'price':
                data.append((string.split(' '),1))
            if json_data[i]["aspect"] == 'service':
                data.append((string.split(' '),2))
            if json_data[i]["aspect"] == 'ambience':
                data.append((string.split(' '),3))
            if json_data[i]["aspect"] == 'misc':
                data.append((string.split(' '),4))
    return data
def load_testdata(fpath):
    data = []
    with open(fpath) as json_file:
        with open('./dataset/acsa_test.json') as s_file:
            s_data = json.load(s_file)
            json_data = json.load(json_file)
            for i in range(0,len(json_data)):
                for j in range(0,len(s_data)):
                    if json_data[i]["sentence"] == s_data[j]["sentence"]:
                        string = json_data[i]["sentence"]
                        string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
                        string = re.sub(r"\'s", " \'s", string)
                        string = re.sub(r"\'ve", " \'ve", string)
                        string = re.sub(r"n\'t", " n\'t", string)
                        string = re.sub(r"\'re", " \'re", string)
                        string = re.sub(r"\'d", " \'d", string)
                        string = re.sub(r"\'ll", " \'ll", string)
                        string = re.sub(r",", " , ", string)            
                        string = re.sub(r"!", " ! ", string)
                        string = re.sub(r"\(", " \( ", string)
                        string = re.sub(r"\)", " \) ", string)
                        string = re.sub(r"\?", " \? ", string)
                        string = re.sub(r"\s{2,}", " ", string)
                        string = string.rstrip()
                        data.append((string.split(' '),json_data[i]["aspect"]))
                        break
    return data

train_data = load_data('./dataset/acsa_train.json')
test_data = load_data('./dataset/acsa_test.json')
data = train_data+test_data

In [3]:
max_sentence_len = max([len(sentence) for sentence, _ in train_data])

vocab = []

for d, _ in data:
    for w in d:
        if w not in vocab:
            vocab.append(w)
vocab = sorted(vocab)
vocab_size = len(vocab)

w2i = {w:i for i,w in enumerate(vocab)}
i2w = {i:w for i,w in enumerate(vocab)}
word_vectors = KeyedVectors.load_word2vec_format('../sentence/dataset/GoogleNews-vectors-negative300.bin', binary=True)
wv_matrix = []
for i in range(len(vocab)):
            word = i2w[i]
            if word in word_vectors.vocab:
                wv_matrix.append(word_vectors.word_vec(word))
            else:
                wv_matrix.append(np.random.uniform(-0.01, 0.01, 300).astype("float32"))

wv_matrix.append(np.random.uniform(-0.01, 0.01, 300).astype("float32"))
wv_matrix.append(np.zeros(300).astype("float32"))
wv_matrix = np.array(wv_matrix)

In [4]:
random.shuffle(train_data)
#random.shuffle(test_data)

In [5]:
class Net(nn.Module):
    def __init__(self,vocab_size,embd_size,out_chs,filter_heights,pretrained_vec):
        super(Net,self).__init__()
        self.embedding = nn.Embedding(vocab_size+2,embd_size)
        self.embedding.weight.data.copy_(torch.from_numpy(pretrained_vec))
        self.conv = nn.ModuleList([nn.Conv2d(1,out_chs,(fh,embd_size),padding = (fh-1,0)) for fh in filter_heights])
        self.dropout = nn.Dropout(.5)
        self.fc1 = nn.Linear(out_chs*len(filter_heights),5)
        
    def forward(self,x):
        embeding = self.embedding(x)
        x = embeding.unsqueeze(1)
        conved = [F.relu(conv(x)).squeeze(3) for conv in self.conv]
        x = [F.avg_pool1d(i,i.size(2)).squeeze(2) for i in conved]
        features = torch.cat(x,1)
        x = self.dropout(features)
        x = self.fc1(x)
        probs = F.softmax(x)
        return probs, features, conved

In [7]:
def train(model, data, batch_size,n_epoch):
    model.train()
    if use_cuda:
        model.cuda()
    losses = []
    optimizer = torch.optim.Adadelta(model.parameters(),lr = 1)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(n_epoch):
        epoch_loss = 0.0
        random.shuffle(data)
        for i in range(0,len(data)-batch_size,batch_size):
            in_data,labels = [],[]
            for sentence, label in data[i:i+batch_size]:
                index_vec = [w2i[w] for w in sentence]
                pad_len = max(0,max_sentence_len - len(index_vec))
                index_vec +=[0]*pad_len
                index_vec = index_vec[:max_sentence_len]
                in_data.append(index_vec)
                labels.append(label)
            sent_var = Variable(torch.LongTensor(in_data))
            if use_cuda:
                sent_var = sent_var.cuda()
            target_var = Variable(torch.LongTensor(labels))
            if use_cuda:
                target_var = target_var.cuda()
            optimizer.zero_grad()
            probs,features,conved = model(sent_var)
            #print(torch.max(probs,1)[1],target_var)
            loss = criterion(probs,target_var)
            loss.backward()
            optimizer.step()
            #print(loss.data[0])
            epoch_loss += loss.data[0]
        
        print('epoch : {:d},loss : {:.3f}'.format(epoch,epoch_loss))
        losses.append(epoch_loss)
    print('Training avg loss : {:.3f}'.format(sum(losses)/len(losses)))
    
    return model, losses
def test(model,data, n_test,min_sentence_len):
    model.eval()
    f = open("data.txt","w")
    loss = 0
    correct = 0
    params = list(model.parameters())
    num_food = 0
    num_price = 0
    num_ambience = 0
    num_service = 0
    num_misc = 0
    for sentence,label in data[:n_test]:
        if len(sentence) < min_sentence_len:
            continue
        index_vec = [w2i[w] for w in sentence]
        sent_var = Variable(torch.LongTensor([index_vec]))
        if use_cuda:
            sent_var = sent_var.cuda()
        out,features,conved = model(sent_var)
        #print(conved)
        #score = out.data[0][0]
        #pred = 1 if score > .5 else 0
        pred = torch.max(out,1)[1]
        
        if pred == label:
            correct += 1
        #loss += math.pow((label-score),2)
        #print(sentence, pred, label)
        #print(conved[0][0][0])
        #print(params[-2][pred][0][0])
        word_weight=[]
        for i in range(0,300):
            if i<100:
                for start in range(2,len(conved[0][0][0])):
                    conved[0][0][i][start] = conved[0][0][i][start]*params[-2][pred][0][i]
            if i>=100 and i<200:
                for start in range(3,len(conved[1][0][0])):
                    conved[1][0][i-100][start] = conved[1][0][i-100][start]*params[-2][pred][0][i]
            if i>=200 and i<300:
                for start in range(4,len(conved[2][0][0])):
                    conved[2][0][i-200][start] = conved[2][0][i-200][start]*params[-2][pred][0][i]
        for length in range(0,len(sentence)):
            sum = 0
            for i in range(0,300):
                if i<100:
                    sum = sum + conved[0][0][i][length+2]
                if i>=100 and i<200:
                    sum = sum + conved[1][0][i-100][length+3]
                if i>=200 and i<300:
                    sum = sum + conved[2][0][i-200][length+4]
            word_weight.append(sum)
        #np_ww = np.asarray(word_weight)
        np_ww_sm = F.softmax(torch.FloatTensor(word_weight))
        np_ww_sm_list = np_ww_sm.tolist()
        for i in range(0,len(sentence)):
            np_ww_sm_list[i] = round(np_ww_sm_list[i],6)
            if np_ww_sm_list[i] >= 0.5:
                np_ww_sm_list[i] = 1
            else:
                np_ww_sm_list[i] = 0
        string1=""
        string2=""
        string3 = ""
        for i in range(0,len(sentence)):
            string1 = string1+sentence[i]
            string1 = string1 + " "
            
        
            np_ww_sm_list[i] = round(np_ww_sm_list[i],6)
            if np_ww_sm_list[i] >= 0.5:
                np_ww_sm_list[i] = 1
            else:
                np_ww_sm_list[i] = 0
                
            string2 = string2+str(np_ww_sm_list[i])
            string2 = string2 + " "
            
            word_weight[i] = word_weight[i].cpu().data.numpy()
            word_weight[i] = word_weight[i].tolist()
            word_weight[i] = round(word_weight[i],6)
            string3 = string3+str(word_weight[i])
            string3 = string3+ " "
            
        if label == 0:
            print('label : {:s}'.format('food'))
            data = "food"
            num_food = num_food+1
            f.write(data+"\n")
        if label == 1:
            print('label : {:s}'.format('price'))
            data = "price"
            num_price = num_price+1
            f.write(data+"\n")
        if label == 2:
            print('label : {:s}'.format('service'))
            data = "service"
            num_service = num_service+1
            f.write(data+"\n")
        if label == 3:
            print('label : {:s}'.format('ambience'))
            data = "ambience"
            num_ambience = num_ambience+1
            f.write(data+"\n")
        if label == 4:
            print('label : {:s}'.format('misc'))
            data = "misc"
            num_misc = num_misc+1
            f.write(data+"\n")
        if pred == 0:
            print('pred : {:s}'.format('food'))
            data = "food"
            f.write(data+"\n")
        if pred == 1:
            print('label : {:s}'.format('price'))
            data = "price"
            f.write(data+"\n")
        if pred == 2:
            print('label : {:s}'.format('service'))
            data = "service"
            f.write(data+"\n")
        if pred == 3:
            print('label : {:s}'.format('ambience'))
            data = "ambience"
            f.write(data+"\n")
        if pred == 4:
            print('label : {:s}'.format('misc'))
            data = "misc"
            f.write(data+"\n")
        f.write(str(len(sentence))+"\n")
        #print(sentence)
        #print(word_weight)
        print(string1)
        f.write(string1+"\n")
        #print(np_ww_sm_list)
        print(string2)
        f.write(string3+"\n")
        f.write(string2+"\n\n\n")
        print('')
    print('Test acc : {:.3f} ({:d}/{:d})'.format(correct/n_test,correct,n_test))
    f.write(str(num_food)+"\n")
    f.write(str(num_price)+"\n")
    
    f.write(str(num_service)+"\n")
    f.write(str(num_ambience)+"\n")
    f.write(str(num_misc)+"\n")
    f.close()
    #print('Test loss : {:.3f}'.format(loss/n_test))
    
out_ch = 100
embd_size = 300
batch_size = 50
n_epoch = 150
filter_size = [3,4,5]
print('filter : ',filter_size)
model = Net(vocab_size,embd_size,out_ch,filter_size,wv_matrix)
model,losses = train(model,train_data,batch_size,n_epoch)
test(model,test_data,len(test_data),max(filter_size))
print('')

filter :  [3, 4, 5]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch : 0,loss : 115.767
epoch : 1,loss : 112.794
epoch : 2,loss : 111.837
epoch : 3,loss : 110.062
epoch : 4,loss : 107.328
epoch : 5,loss : 104.893
epoch : 6,loss : 102.756
epoch : 7,loss : 101.499
epoch : 8,loss : 100.759
epoch : 9,loss : 99.925
epoch : 10,loss : 99.600
epoch : 11,loss : 99.132
epoch : 12,loss : 98.881
epoch : 13,loss : 98.728
epoch : 14,loss : 98.467
epoch : 15,loss : 98.321
epoch : 16,loss : 98.220
epoch : 17,loss : 97.961
epoch : 18,loss : 97.901
epoch : 19,loss : 97.686
epoch : 20,loss : 97.704
epoch : 21,loss : 97.549
epoch : 22,loss : 97.621
epoch : 23,loss : 97.358
epoch : 24,loss : 97.360
epoch : 25,loss : 97.160
epoch : 26,loss : 97.006
epoch : 27,loss : 96.978
epoch : 28,loss : 96.990
epoch : 29,loss : 96.957
epoch : 30,loss : 96.798
epoch : 31,loss : 96.805
epoch : 32,loss : 96.758
epoch : 33,loss : 96.594
epoch : 34,loss : 96.554
epoch : 35,loss : 96.515
epoch : 36,loss : 96.414
epoch : 37,loss : 96.453
epoch : 38,loss : 96.408
epoch : 39,loss : 96.376
e

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


label : food
pred : food
the bread is top notch as well 
0 1 0 0 0 0 0 

label : service
label : misc
i have to say they have one of the fastest delivery times in the city 
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

label : food
pred : food
food is always fresh and hot ready to eat ! 
1 0 0 0 0 0 0 0 0 0 

label : food
pred : food
did i mention that the coffee is outstanding \? 
0 0 1 0 0 0 0 0 0 

label : food
pred : food
certainly not the best sushi in new york , however , it is always fresh , and the place is very clean , sterile 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 

label : ambience
pred : food
certainly not the best sushi in new york , however , it is always fresh , and the place is very clean , sterile 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 

label : misc
pred : food
i trust the people at go sushi , it never disappoints 
0 0 0 0 0 0 1 0 0 0 0 

label : food
pred : food
straight forward , no surprises , very decent japanese food 
0 0 0 0 0 0 0 0 1 0 

label : food
pred

...

label : food
pred : food
it 's a basic pizza joint , not much to look at , but the pizza is what i go for 
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 

label : ambience
label : service
tables are close , so you better be comfortable bumping elbows with other patrons 
1 0 0 0 0 0 0 0 0 0 0 0 0 0 

label : misc
label : misc
was her monday for lunch was working on a film in the area and found this rare jewel 
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 

label : ambience
label : service
not too much so , but enough that there 's a great scene 
1 0 0 0 0 0 0 0 0 0 0 0 0 

label : misc
label : ambience
you feel like you made it to new york when you eat here 
1 0 0 0 0 0 0 0 0 0 0 0 0 

label : misc
label : misc
we 'd walked by it dozens of times and finally settled on a monday night 
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 

label : misc
label : ambience
sum it do n't read the '10 's' reviews , try it to judge for yourself so you wo n't have as hi expectations , then you 'll be sure to enjoy it 
0 0 0 0 0 0 0

label : ambience
label : misc
alias is warm and cozy and you do n't get that squished in feeling you get at 71 clinton 
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 

label : food
label : service
i have never had a bad meal \( or bad service \) pigalle 
0 0 0 0 0 0 0 0 0 1 0 0 0 

label : service
label : service
i have never had a bad meal \( or bad service \) pigalle 
0 0 0 0 0 0 0 0 0 1 0 0 0 

label : food
pred : food
best hot dogs in the tri state area 
1 0 0 0 0 0 0 0 

label : misc
label : misc
this is a typical example of an establishment in which its owners do not know anything about the restaurant business ! 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 

label : misc
label : misc
i would not recommend meet at all , it is a rip off all the way from the time to step in ! 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 

label : misc
label : misc
went there for a special celebration 
1 0 0 0 0 0 

label : ambience
label : ambience
the restaurant was quiet and intimate 
0 0 1 0 0 0 

label 

label : ambience
label : ambience
it does n't look like much on the outside , but the minute you walk inside , it 's a whole other atmosphere 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 

label : misc
label : misc
i have to say the place was awesome 
1 0 0 0 0 0 0 0 

label : food
pred : food
the ground chickpea soup we sampled as a starter tasted somewhat thin 
0 0 1 0 0 0 0 0 0 0 0 0 

label : food
pred : food
we requested they re slice the sushi , and it was returned to us in small cheese like cubes 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 

label : service
label : service
the service , however , is a peg or two below the quality of food \( horrible bartenders \) , and the clientele , for the most part , are rowdy , loud mouthed commuters \( this could explain the bad attitudes from the staff \) getting loaded for an ac dc concert or a knicks game 
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

label : food
label 

label : ambience
label : misc
who said go when the place is quiet during the day \? 
0 0 1 0 0 0 0 0 0 0 0 0 

label : ambience
label : ambience
can be a bit busy around peak times because of the size 
0 0 0 0 0 0 0 0 0 0 1 0 

label : food
pred : food
i was on jury duty , rode my bike up centre street on my lunch break and came across this great little place with awesome chicken tacos and hibiscus lemonade 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 

label : ambience
pred : food
i was on jury duty , rode my bike up centre street on my lunch break and came across this great little place with awesome chicken tacos and hibiscus lemonade 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 

label : misc
pred : food
i was on jury duty , rode my bike up centre street on my lunch break and came across this great little place with awesome chicken tacos and hibiscus lemonade 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 

label : food
pred : food
good 

label : food
label : service
the new menu has a few creative items , they were smart enough to keep some of the old favorites \( but they raised the prices \) , the staff is friendly most of the time , but i must agree with the person that wrote about their favorite words no , ca n't , sorry , boy , they wo n't bend the rules for anyone 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

label : service
label : service
the new menu has a few creative items , they were smart enough to keep some of the old favorites \( but they raised the prices \) , the staff is friendly most of the time , but i must agree with the person that wrote about their favorite words no , ca n't , sorry , boy , they wo n't bend the rules for anyone 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

label : price
label : service
the new menu ha